# Model Prediction
This notebook implements prediction using sliding window approach.

## Imports and Setup

In [1]:
import os
import joblib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Location

In [2]:
version = '20200509'
data_dir = "../data/"
model_dir = '../models/'
output_dir = "../outputs/probmaps/"
input_file = data_dir + '{}_dataset.csv'.format(version)

images_dir = data_dir + 'images/'
indices_dir = data_dir + 'indices/'
pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'
tmp_dir = data_dir + 'tmp/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

In [ ]:
!gsutil -q -m cp gs://immap-images/20200531/*.tif {images_dir}
!gsutil -q -m cp gs://immap-indices/20200531/*.tif {indices_dir}

## Load Dataset

In [3]:
data = pd.read_csv(input_file).reset_index(drop=True)
features = [column  for column in data.columns[:-3]]
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (1029869, 69)


,B1_2015-2016,B2_2015-2016,B3_2015-2016,B4_2015-2016,B5_2015-2016,B6_2015-2016,B7_2015-2016,B8_2015-2016,B9_2015-2016,B10_2015-2016,...,mndwi_2019-2020,ui_2019-2020,nbi_2019-2020,brba_2019-2020,nbai_2019-2020,mbi_2019-2020,baei_2019-2020,target,uid,area
0,0.15100,0.13225,0.14240,0.16430,0.1899,0.25700,0.2960,0.25510,0.32295,0.0396,...,-0.438625,0.151655,0.209614,0.479385,-0.665204,-0.060444,0.948025,3,39,0
1,0.15100,0.12885,0.13790,0.16185,0.1899,0.25700,0.2960,0.25755,0.32295,0.0396,...,-0.435450,0.151655,0.213526,0.488330,-0.663010,-0.059064,0.952352,3,39,0
2,0.15895,0.13730,0.15185,0.18915,0.2264,0.28555,0.3268,0.28085,0.35740,0.0416,...,-0.447333,0.145385,0.222971,0.473118,-0.661291,-0.060496,0.911748,3,39,0


## Instantiate Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

lr = LogisticRegression(penalty='l1', C=1.0)
rf = RandomForestClassifier(
    n_estimators=800, 
    max_depth=12, 
    min_samples_split=15,
    min_samples_leaf=2,
    random_state=42
)

## Train Model

In [ ]:
model, features = model_utils.train_model(rf, data, num_neg_samples=30000, random_state=SEED)

## Save Model

In [ ]:
filename = model_dir + 'model_RF_30k.sav'
joblib.dump(model, filename)

## Load Model

In [3]:
model_names = ['LR_30k', 'RF_30k']
models = []
for model_name in model_names:
    filename = '{}model_{}.sav'.format(model_dir, model_name)
    models.append(joblib.load(filename))

## Prediction

In [ ]:
areas = list(set([image.split('_')[0] for image in os.listdir(images_dir)]))
area_dict = geoutils.get_filepaths(areas, images_dir, indices_dir)

for area in areas:
    for model, model_name in zip(models, model_names):
        out_dir = output_dir + model_name + '/'
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        output = '{}{}_{}_{}.tif'.format(
            out_dir, version, area, model_name
        )
        if not os.path.isfile(output):
            geoutils.get_preds_windowing(
                area=area, 
                area_dict=area_dict,
                model=model, 
                tmp_dir=tmp_dir,
                best_features=features,  
                output=output, 
                grid_blocks=9,
                threshold=0
            )

## Get Ensemble

In [4]:
areas = list(set([image.split('_')[0] for image in os.listdir(images_dir)]))
for area in areas:
    filename1 = '{0:}{3:}/{1:}_{2:}_{3:}.tif'.format(
        output_dir, version, area, model_names[0]
    )
    filename2 = '{0:}{3:}/{1:}_{2:}_{3:}.tif'.format(
        output_dir, version, area, model_names[1]
    )
    out_dir = output_dir + 'ensembled/'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    output_file = '{}{}_{}.tif'.format(out_dir, version, area)
    geoutils.get_rasters_merged(
        filename1,
        filename2,
        output_file,
        tmp_dir,
        grid_blocks=9
    )

100%|██████████| 81/81 [00:16<00:00,  4.99it/s]


## Upload to GCS

In [ ]:
for model in os.listdir(model_dir):
    filename = model_dir + model
    !gsutil -q cp {filename} gs://immap-models/

In [6]:
for model_name in ['ensembled']: 
    out_dir = output_dir + model_name + '/'
    for filename in os.listdir(out_dir):
        bucket = 'gs://immap-output/{}/{}/'.format(version, model_name)
        !gsutil -q cp {out_dir + filename} {bucket}